### LAB02 - AI Plugins in Semantic Kernel

In this lab, we will learn how to use Semantic Kernel to build native and semantic functions and group them together as AI Plugins. We'll work on the following steps:

1) What are AI Plugins? 
2) Create and add a Semantic Function to a Plugin.
3) Create and add a Native Function to a Plugin.
4) Puting it all together.

#### What are AI Plugins?
As the Copilot concept evolves, it's important to think on ways to extend their capabilities, allowing them to retrieve external information, interact with external services and execute actions on the behavior of the user on a safer way, following Responsible AI principles.

The concept of **Plugins** is an answer to this, as they can act as a "bridge" between Copilots / AI Apps and the digital world. 
![Plugins Overview](images/plugins_overview.png)

With Plugins, it's possible to expand Copilots capabilities and promote interoperability across the industry as Semantic Kernel adopts an open standard for plugins, the [OpenAPI Specification](https://swagger.io/specification/).

Let's get started with the creation of Functions, which will be the base of our AI Plugins. In Semantic Kernel we can think of Plugins as a collection of functions. Let's see in the following sections how we can create functions and group them together as a Plugin. 

#### Create and add a Semantic Function to a Plugin

In the first lesson, we have created and used an [**inline**](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/semantic-functions/inline-semantic-functions?tabs=python) Semantic Function to categorize a call.However this approach is fine for development and testing, we need a more robust way to ensure we can reuse the functions across projects and rapidly maintain / change the prompts and function parameters as required.

Therefore, we'll now explore how we can reuse such functions. Semantic Kernel allows us to import files to define functions. We must follow a standard folder structure when creating the files. Here's an example:

```
📁 plugins
│
└─── 📂 plugin_name_A
     |
     └─── 📂 function_name_A 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     |
     └─── 📂 function_name_B 
            |
            └───📄 skprompt.txt
            └───📄 config.json
    📂 plugin_name_B
     |
     └─── 📂 function_name_C 
            |
            └───📄 skprompt.txt
            └───📄 config.json
```

With this structure in mind, we already start organizing our plugins and grouping together functions that belong to a plugin. For each Function (represented by a Folder) we must have two files:
* **skprompt.txt**: it's the file that will contain the prompt of a Semantic Function. Therefore, we can easily maintain our functions by reviewing the prompts;
* **config.json**: it's a configuration file that will describe the function in natural language, define which parameters the function requires, and the overall configuration the LLM should use run that function, such as temperature and maximum number of tokens. 

With this structure in mind, we already undestand how to create a folder structure to contain the funcions for a Call Center solution plugin. In this lesson, we'll recreate the function used before to fit into this structure.

To get started, run the cell below to create a new file called `skprompt.txt` file inside the Semantic Function "categorize" under "CallCenter" plugin. This file represents the Prompt our function will perform and all input parameters that we want to change in runtime, such as the problem description _{{$problem}}_ and the list with the categories the company use _{{$categories}}_.

In [18]:
%%writefile plugins/CallCenterPlugin/categorize/skprompt.txt
You help a Telecom company to classify problems reported by their subscribers. Your role is to provide accurate classification based on problems description and a list of categories that the Telecom company uses.
Classify the problem in one of the provided categories. Only write the output category with no extra text.
Only write one category per problem description.

Examples: 
Problem: My 5G is not working well when I'm in my car.
Categories: Fixed Internet/Wifi, Mobile Internet.
Output Category: Mobile Internet

Problem: My TV is not streaming well from Youtube. It seems that the wifi in my bedroom is weak.
Categories: Mobile Internet, TV, Fixed Internet/Wifi.
Output Category: Fixed Internet/Wifi

Problem: It's impossible to work today, my internet here in my home is so slow!
Categories: Landiline, Mobile Internet, TV, Fixed Internet/Wifi.
Output Category: Fixed Internet/Wifi

Problem: {{$input}}
Categories: {{$categories}}
Output Category: 

Overwriting plugins/CallCenterPlugin/categorize/skprompt.txt


Next, we'll review the `config.json` file to ensure it correctly describes our function. This file will define how the model should execute our prompt. Notice the parameters such as the number of tokens, temperature and description of the function and input parameters.

Run the cell below to create the file.

In [19]:
%%writefile plugins/CallCenterPlugin/categorize/config.json
{
    "schema": 1,
    "type": "completion",
    "description": "Classify a problem report in a call to the Telecom company based on provided categories.",
    "completion": {
         "max_tokens": 10,
         "temperature": 0.5,
         "top_p": 0.0,
         "presence_penalty": 0.0,
         "frequency_penalty": 0.0
    },
    "input": {
         "parameters": [
              {
                   "name": "input",
                   "description": "The problem reported by the user that needs to be classified. Required parameter.",
                   "defaultValue": ""
              },
              {
                   "name": "categories",
                   "description": "A set of categories used by the Telecom company to classify the problems. Required parameter.",
                   "defaultValue": ""
              }
         ]
    }
}

Writing plugins/CallCenterPlugin/categorize/config2.json


At this moment, your Plugins folder should have _at least_ the structure below.
```
📁 plugins
│
└─── 📂 CallCenterPlugin
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
```

Let's define a variable to represent our Plugins folder. It'll be refered later to Import the plugins.

In [1]:
#Define where the plugins are stored. If your plugins are in a different directory, change this parameter.
plugins_directory = "../plugins"

To manipulate functions, we need to ensure we have all our pre-requirements set, as we did in the previous lesson. 

In [2]:
import semantic_kernel as sk

#Importing the Azure Text Completion service connector
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

# Initialize the kernel
kernel = sk.Kernel()

#Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env() 


#Since our function is a Text Completion, we need to add it to the kernel
kernel.add_text_completion_service("Text Completion", 
                                   AzureTextCompletion(deployment, endpoint, api_key))

Now we can refer to our Plugin and import it to be able to use its functions.

In both **config.json** and **skprompt.txt** files we have defined input parameters that "**Categorize**" function requires. In Semantic Kernel we need to use **ContextVariables** to bind values with such parameters. 

After defining the input variables, we can finally call our semantic function available in our Plugin to perform the expected operation. Please note in the code that we can have multiple functions in a Plugin, so in this case we had to explicitly mention which semantic function we would like to use.

In [7]:
# Importing the callcenter Plugin from the plugins directory, so we can start using its functions 
callcenter_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "CallCenterPlugin"
)

#Defining the input variables for the function
var_category = sk.ContextVariables()
var_category["input"] = "My 4G is terrible slow. It's not loading even simple webpages in my mobile."
var_category["categories"] = "Landline, Mobile Internet, TV, Fixed Internet/Wifi, Billing"

#Call the plugin with input variables
category = await kernel.run_async(
    callcenter_plugin["categorize"],
    input_vars = var_category
    )

#Print the output
print(f"Category of the problem: {category}")

Category of the problem:  Mobile Internet


#### Use Semantic Kernel Tools to add a new Semantic Function

Another way to create Semantic Functions is to use [**Sematic Kernel Tools**](https://devblogs.microsoft.com/semantic-kernel/semantic-kernel-tools/), which facilitates the process and simplify testing of semantic functions and other tasks in Semantic Kernel.


 We'll demonstrate how to use it to create a Semantic Function. Later, you can also refer to this [tutorial](https://learn.microsoft.com/en-us/semantic-kernel/vs-code-tools/).

 Alternatively, you can run the cells below to create the "Summarize" function.

In [25]:
%%writefile ../plugins/CallCenterPlugin/summarize/skprompt.txt
Summarize the problem described below. 
Be brief, yet informative.
Try to capture in the summary:
- The problem description;
- The product customer is complaining (if available);
- How agent handled the call;
- The outcome of the call. 

[INPUT]
{{$input}}
[END INPUT]

Writing plugins/CallCenterPlugin/summarize/skprompt2.txt


In [8]:
%%writefile ../plugins/CallCenterPlugin/summarize/config.json
{
    "schema": 1,
    "type": "completion",
    "description": "Summarize a problem recorded from a call center of a Telecom company",
    "completion": {
        "max_tokens": 250,
        "temperature": 0.6,
        "top_p": 0,
        "presence_penalty": 0,
        "frequency_penalty": 0
    },
    "input": {
        "parameters": [
            {
                "name": "input",
                "description": "A transcription of a call center conversation. Required parameter.",
                "defaultValue": ""
            }
        ]
    },
    "default_backends": []
}

Writing ../plugins/CallCenterPlugin/summarize/config2.json


At this point, ensure your Plugins folder structure is as follows, as we'll require both Semantic Functions later in this session.

```
📁 plugins
│
└─── 📂 CallCenterPlugin
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     └─── 📂 summarize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
```

#### Create and add a Native Function to a Plugin
##### Step 1 - Concept

When we think about the concept of AI Apps it's important to remember that an application will certainly require capabilities that will not be provided solely by Large Language Models. For example, performing calculations, retrieving data from a database and even executing API calls are some of the expected capabilities an application generally has. 

Therefore, we still need to run native code and be able to combine it with AI capabilities to go beyond and create transformative applications powered by modern AI!

In semantic Kernel we can add native code in different languages, like C# and Python, to be part of our Plugins. We should use the same folder structure from before, however, instead of prompts and configurations, we will have native code, like .py (Python) or .cs (C#) files.

For example, let's assume that we want to add a new function to "CallCenter" Plugin to transcribe audio files using. Let's see how our folder structure would look like:

```
📁 plugins
│
└─── 📂 CallCenterPlugin
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     |
     |
     └───📄 Transcribe.py
```

##### Step 2 - Transform an existing code into a Native Function for Semantic Kernel
Let's assume we already have a base code in Python that is capable of transcribing an audio file to text, as shown below:

```python
import azure.cognitiveservices.speech as speechsdk

# Define the transcribe_audio function
def transcribe_audio(audio_file, language, subscription_key, region):
    # Set up the speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
    # Define the language of the audio file
    speech_config.speech_recognition_language = language

    # Open the audio file
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file)

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    # Define global variables
    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]

    # Define the stop callback function
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    # Define the recognize callback function
    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    speech_recognizer.session_stopped.connect(stop_cb)
 
    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    # Stop continuous speech recognition
    speech_recognizer.stop_continuous_recognition()
    
    # Convert to string and return the transcription
    return recognized_text_list
```



 We need to do some small changes in the code to ensure it can become a native function and can be addedd to our Plugin in Semantic Kernel. In summary:
- **Transform it in a class:** all native functions must be defined as public methods that belong to a class. This class will represent our Plugin.
- **Import the required libraries:** you need to import the _skill_definition_ module to be able to use the decorators and _SKContext_ module to parse multiple input variables.
```python
#Importing Azure Cognitive Services Speech SDK package
import azure.cognitiveservices.speech as speechsdk

#Importing Semantic Kernel packages to define the function and its parameters
from semantic_kernel.skill_definition import (
    sk_function,
    sk_function_context_parameter,
)
#Importing Semantic Kernel package to parse input variables
from semantic_kernel.orchestration.sk_context import SKContext
#Defining the Transcribe class to be used as a plugin to transcribe audio files
class Transcribe:
...
```
- **Add the decorator @sk_function:** the kernel will use this decorator to be able to register this function as the plugin is loaded. The decorator will also describe the function and its input parameter (if only one), as described in the [documentation](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/native-functions/using-the-skfunction-decorator?tabs=python#use-the-skfunction-decorator-to-define-a-native-function). Such information will be later used by Planners to orchestrate Plugins.
- **Add the decorator sk_function_context_parameter:** if your function has multiple input parameters you need to use this decorator to describe the input parameters with their names and description, so the kernel understand how to call the function and orchestrate them in Planners.
```python
@sk_function(
    description="Transcribe an audio file. Suitable for long audio files",
    name="transcribe_audio",
)
@sk_function_context_parameter(
    name="audio_path",
    description="The path to the audio file to transcribe",
)
@sk_function_context_parameter(
    name="language",
    description="The language of the audio file in the format of Locale (BCP-47) (e.g. en-US)",
)
@sk_function_context_parameter(
    name="subscription_key",
    description="The subscription key for the Azure AI Speech resource",
)
@sk_function_context_parameter(
    name="region",
    description="The region of the Azure AI Speech resource",
)         
def transcribe_audio(self, context: SKContext) -> str:
```
- **Use SKContext to handle input parameters:** if your function requires multiple input parameters, you need to add _SKContext_ as the input of your function and map the individual parameters inside the function. See more details [here](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/native-functions/multiple-parameters?tabs=python#using-context-parameters-to-pass-multiple-inputs).
```python
    #Obtaining and parsing input parameters from SKContext
    audio_path = context["audio_path"]
    language = context["language"]
    subscription_key = context["subscription_key"]
    region = context["region"]
```

Let's review our modified code directly in the **["transcribe.py"](./plugins/CallCenterPlugin/Transcribe.py)** file in the plugins directory. If the file is not available, you can run the file below to create it.

In [ ]:
%%writefile ../plugins/CallCenterPlugin/Transcribe.py
import time

#Importing Azure Cognitive Services Speech SDK package
import azure.cognitiveservices.speech as speechsdk

#Importing Semantic Kernel packages to define the function and its parameters
from semantic_kernel.skill_definition import (
    sk_function,
    sk_function_context_parameter,
)
#Importing Semantic Kernel package to parse input variables
from semantic_kernel.orchestration.sk_context import SKContext

#Defining the Transcribe class to be used as a plugin to transcribe audio files
class Transcribe:
    @sk_function(
        description="Transcribe an audio file. Output the text transcription of the audio",
        name="transcribe_audio",
    )
    @sk_function_context_parameter(
        name="input",
        description="The path in disk to the audio file to transcribe. Required parameter",
    )
    @sk_function_context_parameter(
        name="language",
        description="The language of the audio file in the format of Locale (BCP-47) (e.g. en-US). Required parameter",
    )
    @sk_function_context_parameter(
        name="subscription_key",
        description="The subscription key for the Azure AI Speech resource. Required parameter.",
    )
    @sk_function_context_parameter(
        name="region",
        description="The region of the Azure AI Speech resource. Required parameter.",
    )         
    def transcribe_audio(self, context: SKContext) -> str:
        #Obtaining and parsing input parameters from SKContext
        audio_path = context["input"]
        language = context["language"]
        subscription_key = context["subscription_key"]
        region = context["region"]

        #Set up the speech configuration
        speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
        # Define the language of the audio file
        speech_config.speech_recognition_language = language

        # Open the audio file
        audio_config = speechsdk.audio.AudioConfig(filename=audio_path)

        # Create a speech recognizer
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

        # Define global variables to control the continuous recognition loop and accumulate text
        global done 
        done = False
        global recognized_text_list 
        recognized_text_list=[]

        # Define the stop callback function
        def stop_cb(evt: speechsdk.SessionEventArgs):
            """callback that signals to stop continuous recognition upon receiving an event `evt`"""
            print('CLOSING on {}'.format(evt))
            global done
            done = True

        # Define the recognize callback function
        def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
            """callback for recognizing the recognized text"""
            global recognized_text_list
            recognized_text_list.append(evt.result.text)

        # Connect callbacks to the events fired by the speech recognizer
        speech_recognizer.recognized.connect(recognize_cb)
        speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
        speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
        speech_recognizer.session_stopped.connect(stop_cb)
    
        # Start continuous speech recognition
        speech_recognizer.start_continuous_recognition()
        print("Continuous speech recognition started. Waiting to complete transcription...")
        while not done:
            time.sleep(.1)

        #Stop continuous speech recognition
        speech_recognizer.stop_continuous_recognition()
        
        print("Transcription complete.")
                
        return str(recognized_text_list)

##### Step 3 - Calling a Native Function with Semantic Kernel
Now that we already have created our native function that's capable of Transcribing an audio file, let's see how can we import and use it.

The process is similar to Semantic Functions, but there's a change on how we import each of them. The native functions are imported as "Plugins" following the hierarchy and folder structure we already explored. Once we have the plugin ready as part of ou Import statement, we can use the kernel to load it using the name of the Class.

Finally, we'll define the input parameters our function requires as our Context and call the function using the kernel to run it. 

In [3]:
# Add plugins path to SysPath to ensure they can be imported by the kernel
#Importing the native function from the plugin
import sys
sys.path.append(plugins_directory)

#Importing the plugin
from CallCenterPlugin.Transcribe import Transcribe

#Loading the .env file to use Azure AI Speech
import os
from dotenv import load_dotenv

#Load sensitive data from .env file to use Azure AI Speech
load_dotenv()

#Import the call center plugin to use the transcription
transcribe_fn = kernel.import_skill(Transcribe(), skill_name="CallCenterPlugin")

#Defining the input variables for the transcription funcion
var_transcribe = sk.ContextVariables()
var_transcribe["audio_path"] = "./audio/enUS_long_network_interference.wav"#english_billing_process_sample.wav"
var_transcribe["language"] = "en-US"
var_transcribe["subscription_key"] = os.getenv("AZURE_AI_SPEECH_KEY")
var_transcribe["region"] = os.getenv("AZURE_AI_SPEECH_REGION")

#Call the plugin with input variables
transcription = await kernel.run_async(
    transcribe_fn["transcribe_audio"],
    input_vars = var_transcribe
    )

print(f"Transcription: {transcription.result}")

STT SESSION STARTED: SessionEventArgs(session_id=0403fa0edeb04f5ca6fa201cb6beb98a)
Continuous speech recognition started. Waiting to complete transcription...


#### Puting it all together for a Call Center Analytics solution
Now, let's see a simple example for a Call Center Analytics solution where we:
- Use an audio as input (previous step);
- Use a Native Function to obtain the transcription from the audio (previous step);
- Use a Semantic Function to summarize the problem;
- Use a Semantic Function to category the problem;

 This is one way to chain functions together and manually orchestrate them. In another Lesson, we'll explore how to automate plugins orchestration with Planners.

In [ ]:
#Import the native functions from the Plugin
callcenter_plugin = kernel.import_semantic_skill_from_directory(plugins_directory, "CallCenterPlugin")

#Define the functions that will be used from the plugin
summarize_fn = callcenter_plugin["summarize"]
categorize_fn = callcenter_plugin["categorize"]

#Create the summary from the Transcription, running the Semantic Function
summary = summarize_fn.invoke(transcription.result)

#Define the Summary as the input for the problem categorization and run the Semantic Function
var_category["problem"] = summary.result
category = categorize_fn.invoke(variables = var_category)

#Print the results
print(f"1) Transcription: {transcription.result} \n2) Summary: {summary.result} \n3)Problem Category: {category.result}")